## Import packages

In [1]:
import regex as re
import py_vncorenlp
import pandas as pd
from VietnameseTextNormalizer.ReleasePython3 import VietnameseTextNormalizer
import json

Init Vietnamese Text Normalizer version 1.9.5
	Bùi Tấn Quang - langmaninternet@gmail.com


In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
annotate = py_vncorenlp.VnCoreNLP(annotators=["wseg", "pos", "ner", "parse"], save_dir='./')


2023-12-11 18:36:02 INFO  WordSegmenter:24 - Loading Word Segmentation model
2023-12-11 18:36:02 INFO  PosTagger:23 - Loading POS Tagging model
2023-12-11 18:36:05 INFO  NerRecognizer:34 - Loading NER model
2023-12-11 18:36:18 INFO  DependencyParser:32 - Loading Dependency Parsing model


In [3]:
STOPWORD = "./vietnamese-stopwords-dash.txt"
with open(STOPWORD, 'r', encoding='utf-8') as stop_word_file:
    stop_words = stop_word_file.read().splitlines()
    
ORIGINAL_DATA ="./original_data/"
PREPROCESSED_DATA = "./preprocessed_data/preprocessed_data.json"

## Preprocessing

In [4]:
def preprocess_text(text):
    text= VietnameseTextNormalizer.Normalize(text)
    text = re.sub(r'http\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\b(www\.[^\s]+|(?!https?://)[^\s]+\.[a-z]{2,})\b', '', text)
    # Remove phone numbers
    text = re.sub(r'\b\d{10,}\b', '', text)

    # Remove emails
    text = re.sub(r'\S+@\S+', '', text)
    annotations = annotate.annotate_text(text)
    all_tokens = []
    for sentence_index, sentence_annotations in annotations.items():
        all_tokens.extend(sentence_annotations)
    word_list  = [
        token['wordForm'] for token in all_tokens
        if isinstance(token, dict) and 'wordForm' in token and isinstance(token['wordForm'], str)
        and 'posTag' in token and isinstance(token['posTag'], str) and token['posTag'] in ['N', 'V', 'A']
        and 'nerLabel' in token and token['nerLabel'] not in ['B-PER']
    ]
    clean_words = [word.strip(',').strip().lower() for word in word_list  if word not in stop_words]
    clean_words = [re.sub(r'([^\s\w]|)+', '', sentence) for sentence in clean_words if sentence!='']
    return clean_words


In [5]:

text = """
Chủ tịch NVIDIA nhận định, Chip, AI là chiến lược vô cùng quan trọng, sống còn cho một quốc gia. Ông tin rằng, Việt Nam sẽ là quê hương và trung tâm lớn nhất của NVIDIA.
Việt Nam sẵn sàng đón các nhà đầu tư 

Phát biểu tại tọa đàm xu hướng phát triển ngành công nghiệp bán dẫn, trí tuệ nhân tạo và cơ hội cho Việt Nam do Bộ Kế hoạch và Đầu tư (KH-ĐT) chủ trì, Trung tâm Đổi mới sáng tạo Quốc gia (NIC) và Tập đoàn NVIDIA (Hoa Kỳ) tổ chức, diễn ra Hòa Lạc ngày 11/12, Bộ trưởng Bộ KH-ĐT Nguyễn Chí Dũng cho biết, tuyên bố chung về nâng cấp quan hệ Việt Nam – Hoa Kỳ lên Đối tác Chiến lược Toàn diện đã nêu hợp tác đột phá giữa hai quốc gia trong giai đoạn tới sẽ là đổi mới sáng tạo, công nghệ cao, trong đó có công nghệ bán dẫn và trí tuệ nhân tạo (AI).

Thời gian vừa qua, Việt Nam đã tích cực chuẩn bị các điều kiện để sẵn sàng đón nhận, hợp tác với các doanh nghiệp (DN), nhà đầu tư của Hoa Kỳ trong lĩnh vực đổi mới sáng tạo, công nghiệp bán dẫn và trí tuệ nhân tạo.
"""


In [6]:
preprocessed_text = preprocess_text(text)
print(preprocessed_text)

['chủ_tịch', 'nhận_định', 'chiến_lược', 'sống_còn', 'quốc_gia', 'ông', 'quê_hương', 'trung_tâm', 'sẵn_sàng', 'đón', 'nhà_đầu_tư', 'phát_biểu', 'toạ_đàm', 'xu_hướng', 'phát_triển', 'ngành', 'công_nghiệp', 'bán_dẫn', 'trí_tuệ_nhân_tạo', 'kế_hoạch', 'đầu_tư', 'chủ_trì', 'trung_tâm', 'đổi_mới', 'sáng_tạo', 'tập_đoàn', 'tổ_chức', 'diễn', 'bộ_trưởng', 'bộ', 'kh-đt', 'tuyên_bố_chung', 'nâng_cấp', 'quan_hệ', 'đối_tác', 'chiến_lược', 'nêu', 'hợp_tác', 'đột_phá', 'quốc_gia', 'giai_đoạn', 'đổi_mới', 'sáng_tạo', 'công_nghệ_cao', 'công_nghệ', 'bán_dẫn', 'trí_tuệ_nhân_tạo', 'thời_gian', 'tích_cực', 'sẵn_sàng', 'đón_nhận', 'hợp_tác', 'doanh_nghiệp', 'nhà_đầu_tư', 'lĩnh_vực', 'đổi_mới', 'sáng_tạo', 'công_nghiệp', 'bán_dẫn', 'trí_tuệ_nhân_tạo']


## Process and save data

In [ ]:
# Hàm để đọc nội dung của file dựa trên định dạng
def read_file(file_path):
    with open(file_path, 'rb') as file:
        # Đọc một số byte đầu tiên để xác định định dạng UTF
        raw_data = file.read(4)

        # Kiểm tra xem có phải là UTF-16 hay không
        if raw_data.startswith(b'\xff\xfe') or raw_data.startswith(b'\xfe\xff'):
            # Định dạng UTF-16
            encoding = 'utf-16'
        else:
            # Định dạng mặc định là UTF-8
            encoding = 'utf-8'

    # Đọc toàn bộ nội dung của file với định dạng đã xác định
    with open(file_path, 'r', encoding=encoding) as file:
        content = file.read()

    return content

In [ ]:
# Tạo một danh sách để lưu trữ dữ liệu
data_list = []

# Lấy danh sách tất cả các tệp trong thư mục gốc
file_names = os.listdir(ORIGINAL_DATA)

# Đếm số lượng files để sử dụng trong thông báo tiến trình
total_files = len(file_names)

# Biến đếm để theo dõi số lượng file đã xử lý
processed_files_count = 0

# Xử lý từng file và thêm vào danh sách
for file_name in file_names:
    file_path = os.path.join(ORIGINAL_DATA, file_name)
    print(file_path)
    # Đọc nội dung file dựa trên định dạng
    content = read_file(file_path)
    # Tiền xử lý văn bản
    processed_text = preprocess_text(content)

    # Trích xuất danh mục từ đường dẫn của file
    category = os.path.basename(ORIGINAL_DATA)

    # Tạo đối tượng lưu trữ dữ liệu
    data = {
        "Processed_Content": processed_text,
        "Category": category
    }

    # Thêm vào danh sách
    data_list.append(data)
    
    

    # Tăng biến đếm
    processed_files_count += 1

    # Hiển thị thông báo tiến trình
    print(f"Processed {processed_files_count}/{total_files} files.", end='\r')
    # Ghi dữ liệu vào tệp JSON ngay sau khi xử lý xong mỗi file
    with open(PREPROCESSED_DATA, 'w', encoding='utf-8') as json_file:
        json.dump(data_list, json_file, ensure_ascii=False)
print("\nXử lý và lưu dữ liệu thành công.")

In [8]:
# Đường dẫn đến file JSON
json_file_path = PREPROCESSED_DATA

# Load dữ liệu từ file JSON vào DataFrame
df = pd.read_json(json_file_path, encoding='utf-8', lines=True)

# Hiển thị DataFrame
print(df.tail())

                                        Processed_Content  Category
114317  [hành_trình, giới_tính, bác_sĩ, trái, chỉnh_sử...  Suc khoe
114318  [khi, nàng, ham_muốn, phụ_nữ, tự_động, giản_đơ...  Suc khoe
114319  [khẳng_định, cúm_gà, thúc_giục, quốc_gia, thàn...  Suc khoe
114320  [bệnh_nhân, nhiễm, chiều, viện, vệ_sinh, dịch_...  Suc khoe
114321  [tập_thể_dục, tập, tối, đốt, calo, tập, trao_đ...  Suc khoe


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114322 entries, 0 to 114321
Data columns (total 2 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Processed_Content  114322 non-null  object
 1   Category           114322 non-null  object
dtypes: object(2)
memory usage: 1.7+ MB
